# BH/NS Bianry Sampling from the COMPAS Dataset

## COMPAS
* HP (https://compas.science/)
* GitHub (https://compas.readthedocs.io/en/latest/)
* Patameter List (https://compas.readthedocs.io/en/latest/pages/User%20guide/COMPAS%20output/standard-logfiles-record-specification.html)
    * Stellar Properties (https://compas.readthedocs.io/en/latest/pages/User%20guide/COMPAS%20output/standard-logfiles-record-specification-stellar.html)
    * Binary Properties (https://compas.readthedocs.io/en/latest/pages/User%20guide/COMPAS%20output/standard-logfiles-record-specification-binary.html)

In [1]:
import numpy as np
import pickle
import sys
import h5py as h5
from astropy.table import  unique, vstack, Table, Column, join

## Constant Parameters

In [11]:
# physical constants
G = 6.67428e-8 # Gravitational Constant (dyn cm^2 / g^2)
c = 2.99792458e10 # Light Speed (cm/s)

# mass [g]
mp =  1.673e-24 # Proton Mass
me = 9.1093837015e-28 # Electron Mass
 
# solar quantities
sigma = 5.67e-5 #Stefan-Boltzmann Constant (erg/cm^2/K^4/s)
Msun = 1.99e33 # Solar Mass [g]
Lsun = 3.839e33 # Solar Luminosity [erg/s]
Rsun = 6.955e10 # Solar Radius [cm]
Tsun = (Lsun / (4*np.pi*Rsun**2*sigma))**0.25 # Solar Photospheric Temperature [K]
Zsun = 0.014 # Solar Metallicity

# time [s]
hr = 3600.0 # Hour
day = 24.0*hr # Day
yr =  365.25*day # Year
Myr = 1e6*yr # Mega Year
Gyr = 1e9*yr # Giga Year

# distance [cm]
pc = 3.086e18 # Parsec
Mpc = 1e6*pc # Mega Parsec
Gpc = 1e9*pc # Giga Parsec

q = 4.80320425e-10 
r_e = (q**2/(me*c**2))
sigma_T = (8.*np.pi/3.) * r_e**2

# frequency [Hz]
MHz = 1e6 # Mega Hertz
GHz = 1e9 # Giga Hertz

# radiation [erg/s/cm^2/Hz]
Jy = 1e-23 # Jansky
mJy = 1e-6 * Jy # Milli Jansky
uJy = 1e-9 * Jy # Micro Jansky

## Path Setting

In [3]:
COMPAS_result_dir = '/home/jovyan/COMPAS_02.35.02_N1e7_Fiducial_AllDCO_AIS/'
pathToData = COMPAS_result_dir+'MainRun/COMPAS_Output_wWeights.h5'

# Read data and put in astropy table
File        = h5.File(pathToData ,'r')

SYS_keys  = ['SEED']
             # , 'mixture_weight', 'Mass@ZAMS(1)', 'Mass@ZAMS(2)', 'Metallicity@ZAMS(1)', 'CE_Event_Counter', 
             # 'Merger', 'Optimistic_CE', 'Immediate_RLOF>CE']
RLOF_keys = ['SEED', 
             'Mass(1)<MT', 'Mass(1)>MT', 'Mass(2)<MT', 'Mass(2)>MT', 
             'Stellar_Type(1)<MT', 'Stellar_Type(1)>MT', 'Stellar_Type(2)<MT', 'Stellar_Type(2)>MT', 
             'Luminosity(1)', 'Luminosity(2)', 'Teff(1)', 'Teff(2)', 
             'Radius(1)<MT', 'Radius(2)<MT', 'Radius(1)>MT', 'Radius(2)>MT',
             'SemiMajorAxis<MT', 'SemiMajorAxis>MT', 'Time<MT', 'Time>MT', 
             'CEE>MT', 'MT_Event_Counter']
CE_keys   = ['SEED', 'CE_Event_Counter', 'Merger'] 

SYS = Table()
for key in SYS_keys:
    SYS[key] = File['BSE_System_Parameters'][key][()]

RLOF = Table()
for key in RLOF_keys:
    RLOF[key] = File['BSE_RLOF'][key][()]

CE = Table()
for key in CE_keys:
    CE[key] = File['BSE_Common_Envelopes'][key][()]
    
File.close()

## Select BH or NS binary

### Stellar Types (see https://ui.adsabs.harvard.edu/abs/2000MNRAS.315..543H/abstract)
* 0: Main Sequence (MS) M < 0.7Msun or fully convective
* 1: MS M > 0.7Msun
* 2: Hertzsprung Gap (HG)
* 3: First Giant Branch
* 4: Core Helium Burning (CHeB)
* 5: Early Asymptotic Giant Branch (EAGB)
* 6: Thermally Pulsing AGB
* 7: Nakid He Star
* 8: Nakid He Star Hertzsprung Gap
* 9: Nakid He Star Giant Branch
* 10: He White Dwarf (He WD)
* 11: C/O WD
* 12: O/Ne WD
* 13: Neutron Star (NS)
* 14: Black Hole (BH)
* 15: massless remnant

In [4]:
RLOF_BH_bool = ((np.logical_and(RLOF['Stellar_Type(1)<MT'] == 14, RLOF['Stellar_Type(2)>MT'] < 13)) |
                (np.logical_and(RLOF['Stellar_Type(2)<MT'] == 14, RLOF['Stellar_Type(1)>MT'] < 13)))

RLOF_NS_bool = ((np.logical_and(RLOF['Stellar_Type(1)<MT'] == 13, RLOF['Stellar_Type(2)>MT'] < 13)) |
                (np.logical_and(RLOF['Stellar_Type(2)<MT'] == 13, RLOF['Stellar_Type(1)>MT'] < 13)))

RLOF_BH = RLOF[RLOF_BH_bool]
RLOF_NS = RLOF[RLOF_NS_bool]

## Select Stable (RLOF; CEE>MT == 0) or Unstable Mass Transfer (Common Envelope; CEE>MT == 1)

In [5]:
CE_wo_merger = CE[CE['Merger']==False]
seed_CE_wo_merger_dropdup = np.unique(CE_wo_merger['SEED'])


RLOF_BH_MT = RLOF_BH[RLOF_BH['CEE>MT']==0]
RLOF_BH_CE_all = RLOF_BH[RLOF_BH['CEE>MT']==1]

common_seeds_woCE = np.intersect1d(RLOF_BH_CE_all['SEED'], seed_CE_wo_merger_dropdup)
mask = np.isin(RLOF_BH_CE_all['SEED'], common_seeds_woCE)
RLOF_BH_CE = RLOF_BH_CE_all[mask]

# # RLOF_BH_CE_all = join(RLOF_BH_CE_all, CE, keys='SEED', join_type='left')
# # RLOF_BH_CE = RLOF_BH_CE_all[RLOF_BH_CE_all['Merger']==False]

RLOF_NS_MT = RLOF_NS[RLOF_NS['CEE>MT']==0]
RLOF_NS_CE_all = RLOF_NS[RLOF_NS['CEE>MT']==1]

common_seeds_woCE = np.intersect1d(RLOF_NS_CE_all['SEED'], seed_CE_wo_merger_dropdup)
mask = np.isin(RLOF_NS_CE_all['SEED'], common_seeds_woCE)
RLOF_NS_CE = RLOF_NS_CE_all[mask]

# RLOF_NS_CE_all = join(RLOF_NS_CE_all, CE, keys='SEED', join_type='left')
# RLOF_NS_CE = RLOF_NS_CE_all[RLOF_NS_CE_all['Merger']==False]

## Compute Typical Parameters

### Timescale
For stable mass transfer (Roche Lobe Overflow), we assume the accretion timescale ($t_{\rm acc}$) as follows:
$$
t_{\rm acc} \approx t_{\rm Thermal} = \frac{GM^{2}}{2RL} \approx 31 {\rm ~[Myr]}\left(\frac{M}{M_{\odot}}\right)^{2}\left(\frac{R}{R_{\odot}}\right)^{-1}\left(\frac{L}{L_{\odot}}\right)^{-1},
$$
where $M$, $R$, and $L$ are the mass, radius, and luminosity of the donors before the mass transfer derived from COMPAS simulations, respectively.

On the other hand, for unstable mass transfer (Common Envelope), we assume the accretion timescale as follows:
$$
t_{\rm acc} \approx 100\times t_{\rm Orbital} = 2\pi\sqrt{\frac{a^{3}}{G(M_{1} + M_{2})}} \approx 1.6 {\rm ~[yr]}\left(\frac{a}{1000R_{\odot}}\right)^{3/2}\left(\frac{M_1+M_2}{40M_{\odot}}\right)^{-1/2}
$$
where $M_{1}$ and $M_{2}$ are the mass of primery and secondary stars before the mass transfer, respectively, and $a$ is the semi-major axis before the mass transfer.

Note that other tyical timescales of binaries (dynamical timescale ($t_{\rm dyn}$ and nuclear timescale ($t_{\rm nuc}$)) are as follows:
$$
t_{\rm dyn} = \sqrt{\frac{R^{3}}{2GM}} \approx 10^{3} {\rm ~[sec]}~ 
\left(\frac{R}{R_{\odot}}\right)^{3/2}
\left(\frac{M}{M_{\odot}}\right)^{-1/2} \\
t_{\rm nuc, H} = \frac{\frac{\eta X M}{4m_{p}} \times \Delta E_{4H\to He}}{L} = 7{\rm ~[Gyr]}~ 
\left(\frac{\eta}{0.1}\right)
\left(\frac{X}{0.7}\right)
\left(\frac{M}{M_{\odot}}\right)
\left(\frac{\Delta E_{4H\to He}}{26 {\rm ~MeV}}\right)
$$

### Accretion Rate
The Eddington luminosity ($L_{\rm Edd}$) is defined by the balance of the gravity force and radiation pressure:
$$
L_{\rm Edd} = \frac{4\pi c GM m_{p}}{\sigma_{\rm T}} \approx 1.2\times 10^{38} {\rm ~erg/s} \left(\frac{M}{M_{\odot}}\right),
$$
where, $c$ is the light speed, $G$ is the gravitional constant, $m_{p}$ is the proton mass, $\sigma_{\rm T}$ is the cross section of electron scattering, and $M$ is the BH mass.

Then, the Eddington accretion rate is defined as follows:
$$
\dot{M}_{\rm Edd} 
= \frac{L_{\rm Edd}}{\eta c^{2}} 
\approx 1.3 \times 10^{18}{\rm ~g/s} \left(\frac{\eta}{0.1}\right)^{-1}\left(\frac{M}{M_{\odot}}\right)
\approx 2.2 \times 10^{-8}{\rm ~M_{\odot}/yr} \left(\frac{\eta}{0.1}\right)^{-1}\left(\frac{M}{M_{\odot}}\right),
$$
where $\eta$ is the energy conversion efficiency from the accretion rate to luminosity. Here, we assume $\eta = 0.1$, i.e., 10%.

Based on the COMPAS dataset, we defined the accreted mass ($M_{\rm acc}$) from a donor to a compact object as follows:
$$
M_{\rm acc} = (M_{\rm donor,~before~MT} - M_{\rm donor,~after~MT}),
$$
where, $M_{\rm donor}$ and $M_{\rm CO}$ are the masses of the donor and compact object, and the subscripts of 'before/after MT' mean before or after mass transfter.

Finally, the accretion rate ($\dot{M}$) is defined as follows:
$$
\dot{M} = \frac{\dot{M}}{t_{\rm acc}}, 
$$
and the Eddington ratio ($R_{\rm Edd}$) is described as follows:
$$
R_{\rm Edd} = \frac{\dot{M}}{\dot{M}_{\rm Edd}}
$$

In [6]:
def calculate_kh_timescale(L, R, M):
    return G*(M*Msun)**2 / ((L*Lsun)*(R*Rsun)) / Myr

def calculate_dynamical_timescale(R, M):
    return np.sqrt((R*Rsun)**3 / (2*G*(M*Msun))) / Myr

def calculate_circulation_timescale(a, M1, M2):
    return 2*np.pi*np.sqrt((a*Rsun)**3 / (G*((M1+M2)*Msun))) / Myr

def calculate_eddington_ratio(MBH, M_acc, t, eta=0.1):
    Mdot     = M_acc*Msun / (t*Myr)
    L_edd    = 4*np.pi*G*mp*c*MBH*Msun/sigma_T
    Mdot_edd = L_edd/(eta*c**2)
    return Mdot/Mdot_edd

In [7]:
def calculate_parameters(data, CO_type=14, MT_types='RLOF'):
    
    data['Accreted_Mass'] = np.where(data['Stellar_Type(1)<MT']==CO_type,
                                     (data['Mass(2)<MT']-data['Mass(2)>MT']), (data['Mass(1)<MT']-data['Mass(1)>MT']))
    
    data['Timescale_KH']  = np.where(data['Stellar_Type(1)<MT']==CO_type,
                                     calculate_kh_timescale(data['Luminosity(2)'], data['Radius(2)<MT'], data['Mass(2)<MT']),
                                     calculate_kh_timescale(data['Luminosity(1)'], data['Radius(1)<MT'], data['Mass(1)<MT']))
    data['Timescale_Cic']  = calculate_circulation_timescale(data['SemiMajorAxis<MT'], data['Mass(1)<MT'], data['Mass(2)<MT'])
    data['Timescale_dt']  = data['Time>MT'] - data['Time<MT']
    
    if MT_types == 'RLOF':
        data['Timescale'] = data['Time>MT'] - data['Time<MT']
        data['Eddington'] = np.where(data['Stellar_Type(1)<MT']==CO_type,
                                     calculate_eddington_ratio(data['Mass(1)<MT'], data['Accreted_Mass'], data['Timescale']),
                                     calculate_eddington_ratio(data['Mass(2)<MT'], data['Accreted_Mass'], data['Timescale']))
    
    elif MT_types == 'CE':
        data['Timescale'] = 100*calculate_circulation_timescale(data['SemiMajorAxis<MT'], data['Mass(1)<MT'], data['Mass(2)<MT'])
        data['Eddington'] = np.where(data['Stellar_Type(1)<MT']==CO_type,
                                         calculate_eddington_ratio(data['Mass(1)<MT'], data['Accreted_Mass'], data['Timescale']),
                                         calculate_eddington_ratio(data['Mass(2)<MT'], data['Accreted_Mass'], data['Timescale']))
    
    return data

In [12]:
RLOF_BH_MT = calculate_parameters(RLOF_BH_MT, CO_type=14, MT_types='RLOF')
RLOF_BH_CE = calculate_parameters(RLOF_BH_CE, CO_type=14, MT_types='CE')

RLOF_NS_MT = calculate_parameters(RLOF_NS_MT, CO_type=13, MT_types='RLOF')
RLOF_NS_CE = calculate_parameters(RLOF_NS_CE, CO_type=13, MT_types='CE')

/tmp/ipykernel_5365/4143798075.py:2: RuntimeWarning: divide by zero encountered in divide
  return G*(M*Msun)**2 / ((L*Lsun)*(R*Rsun)) / Myr


An Error Message will come up. This error is caused by L=0.0 in the calculation for the KH timescale. I will take this error into account later. See also following MEMO.


##### MEMO

In some Common Envelope cases, the luminosity of donors is zero in the original COMPAS output.
This bug are occured in some AGB stars (Stellar Type = 5).<br>
Thus, we drop the datasets including the bugs (Now, comment out).

In [13]:
# RLOF_BH_CE = RLOF_BH_CE[np.isfinite(RLOF_BH_CE['Timescale'])]
# RLOF_NS_CE = RLOF_NS_CE[np.isfinite(RLOF_NS_CE['Timescale'])]

## Select Super-Eddington Sysmtems ($\dot{M}/\dot{M}_{\rm Edd} > 10 ~\text{or}~ 100$)

In [14]:
def filter_eddington_ratio(data, Eddington_Min=100, Flag_All=False):
    
    if Flag_All==False:
        data = data[data['Eddington']>Eddington_Min].group_by('SEED')
        group_by_Eddington_Max = data['SEED', 'Eddington'].groups.aggregate(np.max)
        data_Eddington = data[np.in1d(data['Eddington'], group_by_Eddington_Max['Eddington'])]
    else:
        data_Eddington = data[data['Eddington']>Eddington_Min]
    
    return data_Eddington

In [15]:
Edd_limit = 100
RLOF_BH_MT_Edd = filter_eddington_ratio(RLOF_BH_MT, Eddington_Min=Edd_limit)
RLOF_BH_CE_Edd = filter_eddington_ratio(RLOF_BH_CE, Eddington_Min=Edd_limit)

RLOF_NS_MT_Edd = filter_eddington_ratio(RLOF_NS_MT, Eddington_Min=Edd_limit)
RLOF_NS_CE_Edd = filter_eddington_ratio(RLOF_NS_CE, Eddington_Min=Edd_limit)

## Check the dataset

In [16]:
def select_donors(data, CO_type=14):
    
    CO1_bool = (data['Stellar_Type(1)<MT'] == CO_type)
    
    Donor2 = data[CO1_bool]
    Donor2_data = Donor2['SEED',
                         'Mass(1)<MT', 'Mass(1)>MT',
                         'Mass(2)<MT', 'Mass(2)>MT',
                         'Stellar_Type(1)<MT', 'Stellar_Type(1)>MT', 
                         'Stellar_Type(2)<MT', 'Stellar_Type(2)>MT', 
                         'Radius(2)<MT', 'Radius(2)>MT',
                         'Luminosity(2)', 'Teff(2)',
                         'SemiMajorAxis<MT', 'SemiMajorAxis>MT', 'Time<MT', 'Time>MT',
                         'Accreted_Mass', 'Timescale_KH', 'Timescale_Cic', 'Timescale_dt',
                         'Timescale', 'Eddington']
    Donor2_data.rename_column('Mass(1)<MT', 'Mass(A)<MT')
    Donor2_data.rename_column('Mass(1)>MT', 'Mass(A)>MT')
    Donor2_data.rename_column('Stellar_Type(1)<MT', 'Stellar_Type(A)<MT')
    Donor2_data.rename_column('Stellar_Type(1)>MT', 'Stellar_Type(A)>MT')
    Donor2_data.rename_column('Mass(2)<MT', 'Mass(D)<MT')
    Donor2_data.rename_column('Mass(2)>MT', 'Mass(D)>MT')
    Donor2_data.rename_column('Stellar_Type(2)<MT', 'Stellar_Type(D)<MT')
    Donor2_data.rename_column('Stellar_Type(2)>MT', 'Stellar_Type(D)>MT')
    Donor2_data.rename_column('Radius(2)<MT', 'Radius(D)<MT')
    Donor2_data.rename_column('Radius(2)>MT', 'Radius(D)>MT')
    Donor2_data.rename_column('Luminosity(2)', 'Luminosity(D)')
    Donor2_data.rename_column('Teff(2)', 'Teff(D)')
    
    
    CO2_bool = (data['Stellar_Type(2)<MT'] == CO_type)
    Donor1 = data[CO2_bool]
    Donor1_data = Donor1['SEED',
                         'Mass(1)<MT', 'Mass(1)>MT',
                         'Mass(2)<MT', 'Mass(2)>MT',
                         'Stellar_Type(1)<MT', 'Stellar_Type(1)>MT', 
                         'Stellar_Type(2)<MT', 'Stellar_Type(2)>MT', 
                         'Radius(1)<MT', 'Radius(1)>MT',
                         'Luminosity(1)', 'Teff(1)',
                         'SemiMajorAxis<MT', 'SemiMajorAxis>MT', 'Time<MT', 'Time>MT',
                         'Accreted_Mass', 'Timescale_KH', 'Timescale_Cic', 'Timescale_dt',
                         'Timescale', 'Eddington']
    Donor1_data.rename_column('Mass(2)<MT', 'Mass(A)<MT')
    Donor1_data.rename_column('Mass(2)>MT', 'Mass(A)>MT')
    Donor1_data.rename_column('Stellar_Type(2)<MT', 'Stellar_Type(A)<MT')
    Donor1_data.rename_column('Stellar_Type(2)>MT', 'Stellar_Type(A)>MT')
    Donor1_data.rename_column('Mass(1)<MT', 'Mass(D)<MT')
    Donor1_data.rename_column('Mass(1)>MT', 'Mass(D)>MT')
    Donor1_data.rename_column('Stellar_Type(1)<MT', 'Stellar_Type(D)<MT')
    Donor1_data.rename_column('Stellar_Type(1)>MT', 'Stellar_Type(D)>MT')
    Donor1_data.rename_column('Radius(1)<MT', 'Radius(D)<MT')
    Donor1_data.rename_column('Radius(1)>MT', 'Radius(D)>MT')
    Donor1_data.rename_column('Luminosity(1)', 'Luminosity(D)')
    Donor1_data.rename_column('Teff(1)', 'Teff(D)')

    return vstack([Donor2_data, Donor1_data])

In [17]:
MT_Edd = vstack([select_donors(RLOF_BH_MT_Edd, CO_type=14), select_donors(RLOF_NS_MT_Edd, CO_type=13)])
CE_Edd = vstack([select_donors(RLOF_BH_CE_Edd, CO_type=14), select_donors(RLOF_NS_CE_Edd, CO_type=13)])

## Save the sampling data as mt_edd000/ce_edd000.pkl

In [18]:
pickle.dump(MT_Edd, open(f'./sampling_data/mt_edd{Edd_limit}.pkl','wb'), protocol=2)
pickle.dump(CE_Edd, open(f'./sampling_data/ce_edd{Edd_limit}.pkl','wb'), protocol=2)